# Music Listening Habits Data Mining

In [1]:
import pandas as pd
import os
import json

## Data needs to support vizualisation

- Top 15 artists (name & link) ✅
- Listening duration per artist ✅
- Number of different tracks listened per artist ✅

- Top 15 genres for each user ✅
- Listening time for each genre ✅
- Proportion of listening per genre ✅

- Average listening time per user ✅

- Ranking of genres per period (each week, each months)✅
- Ranking of tracks per period (each week, each months)✅
- Ranking of artist per period  (each week, each months)✅

## Import Data for each users

In [45]:
df_clement = pd.read_excel("./data/Archive-Clement-Deezer/clement-deezer-data.xlsx", "10_listeningHistory")

In [55]:
df_clement.head(5)

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time,Platform Name,Platform Model,Date
0,Beggin,Madcon,NOBLV0703011,Beggin,37.165.218.4,1,android,HUAWEI ANE-LX1,2022-03-08 13:49:04
1,Beggin,Madcon,NOBLV0703011,Beggin,37.166.33.146,218,android,HUAWEI ANE-LX1,2022-02-22 16:19:01
2,Beggin,Madcon,NOBLV0703011,Beggin,37.166.68.99,218,android,samsung SM-A528B,2023-11-11 11:07:36
3,Beggin,Madcon,NOBLV0703011,Beggin,193.51.159.240,4,android,HUAWEI ANE-LX1,2022-04-20 09:07:38
4,Beggin,Madcon,NOBLV0703011,Beggin,37.169.42.32,218,android,HUAWEI ANE-LX1,2022-02-27 18:53:00


In [54]:
df_clement.count("")

Song Title        36276
Artist            36276
ISRC              36276
Album Title       36276
IP Address        36276
Listening Time    36276
Platform Name     36276
Platform Model    31978
Date              36276
dtype: int64

## Retrieve useful columns

In [18]:
df_clement = df_clement[["Song Title", "Artist", "Album Title", "Date"]]


## Columns format

In [56]:
df_clement = df_clement.sort_values("Date")
df_clement['Date'] = pd.to_datetime(df_clement['Date'])

In [59]:
(df_clement.head
(1))

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time,Platform Name,Platform Model,Date
9366,Mme Pavoshko,Black M,FR7U61200382,Le monde plus gros que mes yeux,88.170.1.13,256,android,NaN,2015-06-21 18:15:59


## Explore within time interval

In [41]:
# Define the date interval
start_date = "2024-01-01"
end_date = "2025-01-31"

# Filter the DataFrame
date_filtered_df = df_clement[(df_clement['Date'] >= start_date) & (df_clement['Date'] <= end_date)]

In [42]:
artist_counts = date_filtered_df.groupby('Artist').size().reset_index(name='Total Count')

In [43]:
artist_counts.sort_values("Total Count", ascending=False).head(15)

,Artist,Total Count
141,Imagine Dragons,292
129,Grandson,261
296,Starset,168
186,Linkin Park,138
313,System of a Down,115
308,Sub Urban,105
347,Three Days Grace,101
131,Green Day,95
137,Hollywood Undead,92
103,Escape the Fate,91
